# RL Exercise 4 - Training and Serving a Policy with Ray

**GOAL:** The goal of this exercise is to show how to train a policy with Ray and to deploy it with Clipper in a fun, interactive way.

We will train an agent to play Pong, and then we will play Pong against the policy that we trained.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
import pong_py
import ray

from ray.rllib.ppo import PPOAgent, DEFAULT_CONFIG

Start Ray.

In [ ]:
ray.init(num_workers=0)

The cell below is a hack. The explanation is as follows. Internally within the `PPOAgent` constructor, a number of actors are created, and these actors will instantiate gym environments using the command `gym.make('PongJS-v0')`. The command `gym.make` knows how to instantiate a number of pre-defined environments that are shipped with the `gym` module. However, the `PongJS-v0` environment is defined in the `pong_py` module and is registered with the `gym` module when the `import pong_py` statement gets run.

Therefore, for the actors to successfully instantiate the gym environments, the `pong_py` module must be imported on the actors. This is why we define a remote function `import_pong_py` which closes over the `pong_py` environment. When the actors are created, that remote function is unpickled on the actors which forces the `pong_py` module to be imported, which enables the `gym` module to create the `PongJS-v0` environment.

In [ ]:
# This is a hack.
@ray.remote
def import_pong_py():
    pong_py

Instantiate an agent that can be trained using Proximal Policy Optimization (PPO).

In [ ]:
config = DEFAULT_CONFIG.copy()
# Consider using more workers to speed up the rollouts.
config['num_workers'] = 3
config['gamma'] = 0.99
config['sgd_stepsize'] = 5e-3
config['kl_coeff'] = 0.1
config['num_sgd_iter'] = 20
config['sgd_batchsize'] = 8196
config['observation_filter'] = 'NoFilter'
config['model']['fcnet_hiddens'] = [32, 32]

agent = PPOAgent('PongJS-v0', config)

Train the `PPOAgent` for some number of iterations.

**EXERCISE:** You will need to experiment with the number of iterations as well as with the configuration to get the agent to learn something reasonable.

In [ ]:
for i in range(2):
    result = agent.train()

Use the agent manually by calling `agent.compute_action` and see the rewards you get are consistent with the rewards printed during the training procedure.

In [ ]:
env = gym.make('PongJS-v0')

for _ in range(20):
    state = env.reset()
    done = False
    cumulative_reward = 0

    while not done:
        action = agent.compute_action(state)
        state, reward, done, _ = env.step(action)
        cumulative_reward += reward

    print(cumulative_reward)

Checkpoint the agent so that the relevant model can be saved and served by the actor webserver.

In [ ]:
checkpoint_path = agent.save()

Checkpoint the agent so that the relevant model can be saved and deployed to Clipper. We save the name of the checkpoint file in `metadata.json` so the model container knows how to restore the policy checkpoint.

In [ ]:
import cgi
from http.server import BaseHTTPRequestHandler, HTTPServer
import json
import socketserver

def make_handler_class(agent):

    class PolicyHandler(BaseHTTPRequestHandler):
        def __init__(self, *args, **kwargs):
            BaseHTTPRequestHandler.__init__(self, *args, **kwargs)
            
        def end_headers(self):
            self.send_header('Access-Control-Allow-Origin', '*')
            self.send_header('Access-Control-Allow-Methods', '*')
            self.send_header('Access-Control-Allow-Headers', 'Content-Type')
            BaseHTTPRequestHandler.end_headers(self)
        
        def do_OPTIONS(self):
            self.send_response(200, 'ok')
            self.end_headers()

        def do_POST(self):
            length = int(self.headers.get_all('content-length')[0])
            post_body = cgi.parse_qs(self.rfile.read(length), keep_blank_values=1)
            features = json.loads(list(post_body.keys())[0])['input']

            self.send_response(200)
            self.send_header('Content-type', 'json')
            self.end_headers()

            action = int(agent.compute_action(features))
        
            self.wfile.write(json.dumps({'output': action}).encode('ascii'))

    return PolicyHandler

@ray.remote
class PolicyServer(BaseHTTPRequestHandler):
    def __init__(self, port, config, checkpoint_path):
        self.port = port

        config['num_workers'] = 0
        agent = PPOAgent('PongJS-v0', config)
        agent.restore(checkpoint_path)

        self.HandlerClass = make_handler_class(agent)

    def run_server(self):
        # run web server, do this in a separate thread
        server_address = ('', self.port)
        httpd = HTTPServer(server_address, self.HandlerClass)
        print('Starting server.')
        httpd.serve_forever()

    def update_policy(self):
        # update policy
        pass


server = PolicyServer.remote(3000, config, checkpoint_path)
server.run_server.remote()

In [ ]:
import os
import json
checkpoint_path = agent.save()
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_file = os.path.basename(checkpoint_path)
with open(os.path.join(checkpoint_dir, "metadata.json"), "w") as f:
    json.dump({"checkpoint": checkpoint_file}, f)

## Play Against the Policy

In this section, we will play Pong against the policy that we just trained. The game will be played in your browser, and the policy that we trained will be served by Clipper.

**EXERCISE:** Deploy your policy using Clipper. Follow the instructions that get printed below to play Pong against the deployed policy. You'll need to deploy all of the data that is saved in the directory `os.path.dirname(checkpoint_path)`.

Start by importing the `clipper_admin` library and use that to create a new Clipper instance to serve the policy.

When you create your ClipperConnection, you need to tell it how to communicate with the Docker service and Clipper. You can use the following command to get the Docker IP address. Use that address when you create your `ClipperConnection` in the next step.

Now you can start the Pong webserver. It will print out the URL it's running on after it starts. Copy and paste that URL into your browser and press "1" to play against your policy!

In [ ]:
import subprocess
server_handle = subprocess.Popen(["./start_webserver.sh", clipper_addr], stdout=subprocess.PIPE)